https://realpython.com/python-descriptors/

https://docs-python.ru/tutorial/klassy-jazyke-python/bazovyj-obzor-deskriptora-klassa/

https://docs.python.org/3/howto/descriptor.html

На этом занятии мы затронем довольно интересную, но не простую тему – дескрипторы. Я начну с одного явного недостатка объектов-свойств (@property), о которых мы с вами уже подробно говорили. Давайте представим, что создаем класс для представления точек в трехмерном пространстве:

```Py
class Point3D:
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z
```
Здесь у нас формируются защищенные локальные свойства для создаваемого объекта класса Point3D. Теперь представим, что согласно заданию координаты должны представляться исключительно целыми числами. Для этого я пропишу следующий метод проверки:
```Py
@classmethod
def verify_coord(cls, coord):
    if type(coord) != int:
        raise TypeError("Координата должна быть целым числом")
```
А вызывать его будут в сеттерах соответствующих свойств класса:
```Py
@property
def x(self):
    return self._x

@x.setter
def x(self, coord):
    self.verify_coord(coord)
    self._x = coord

@property
def y(self):
    return self._y

@y.setter
def y(self, coord):
    self.verify_coord(coord)
    self._y = coord

@property
def z(self):
    return self._z

@z.setter
def z(self, coord):
    self.verify_coord(coord)
    self._z = coord
```

И теперь мы можем в инициализаторе использовать эти объекты-свойства для формирования локальных атрибутов экземпляров:
```Py
class Point3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
```

Все, создавая объекты этого класса:
```Py
p = Point3D(1, 2, 3)
print(p.__dict__)
```

у нас автоматически будут формироваться нужные локальные атрибуты и мы сможем с ними работать через объекты-свойства x, y, z.

In [49]:
class Point3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Координата должна быть целым числом")
        
    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, coord):
        self.verify_coord(coord)
        self._x = coord

    @property
    def y(self):
        return self._y

    @y.setter
    def y(self, coord):
        self.verify_coord(coord)
        self._y = coord

    @property
    def z(self):
        return self._z

    @z.setter
    def z(self, coord):
        self.verify_coord(coord)
        self._z = coord

p = Point3D(1, 2, 3)
print(p.__dict__)

{'_x': 1, '_y': 2, '_z': 3}


Но, смотрите, в нашем классе Point3D получилось своеобразное дублирование: мы три раза прописывали свойства, фактически, с одинаковым функционалом. Менялись только названия методов и локальных атрибутов. Представьте, во что превратится описание этого класса, если нужно будет задать 10 и более таких объектов-свойств! Программист во всем этом просто запутается, да и редактировать такую программу станет непросто. Как можно все это оптимизировать? Здесь нам на помощь как раз и приходят дескрипторы.

Дескрипторы - это объекты Python, реализующие метод протокола дескрипторов, который дает вам возможность создавать объекты с особым поведением, когда к ним обращаются как к атрибутам других объектов. Здесь вы можете увидеть правильное определение протокола дескрипторов:
```Py
__get__(self, obj, type=None) -> object
__set__(self, obj, value) -> None
__delete__(self, obj) -> None
__set_name__(self, owner, name)
```
Для реализации дескриптора классу требуется определить один или несколько методов, входящих в протокол дескриптора.

Если ваш дескриптор реализует только ```.__get__()```, то говорят, что это **non-data descriptor** (дескриптор не данных). Если он реализует ```.__set__()``` или ```.__delete__()```, то говорят, что это **data descriptor** (дескриптор данных). Обратите внимание, что разница не только в названии, но и в поведении. Это потому, что дескрипторы данных имеют приоритет в процессе поиска, как вы увидите позже.

Про OBJECT.```__SET_NAME__```

```object.__set_name__(self, owner, name)  -> None```
 - self : Ссылка на экземпляр дескриптора.

 - owner : Класс владельца дескриптора.

 - name : Имя атрибута из класса владельца, связанное с дескриптором. # a = Descr() # self is link for descriptor's instance, name is "a", owner is link for owner

Один из методов протокола дескриптора. Позволяет отследить имя атрибута, к которому дескриптор привязан в классе.
Появился в Py 3.6
Каждый раз, когда вы инстанцируете дескриптор, вызывается этот метод и автоматически устанавливается параметр name.
`
a = Descr()

Если вы хотите использовать дескрипторы Python в своем коде, то вам просто необходимо реализовать протокол дескрипторов. Наиболее важными методами этого протокола являются ```.__get__()``` и ```.__set__()```, которые имеют следующую сигнатуру:
```Py
__get__(self, obj, type=None) -> object
__set__(self, obj, value) -> None
```
Когда вы реализуете протокол, помните об этих вещах:

- self - это экземпляр дескриптора, который вы пишете.
- obj - это экземпляр объекта, к которому прикреплен ваш дескриптор.
- type - это тип объекта, к которому прикреплен дескриптор.
В ```.__set__()``` у вас нет переменной type, потому что вы можете вызвать ```.__set__()``` только для объекта. В отличие от этого, вы можете вызывать ```.__get__()``` как для объекта, так и для класса.

Когда атрибут класса является дескриптором, при поиске атрибута срабатывает специальное поведение связывания. Обычно при использовании a.b для получения, установки или удаления атрибута происходит поиск объекта с именем b в словаре класса для a, но если b является дескриптором, то вызывается соответствующий метод дескриптора. 

Чтобы использовать дескриптор, он должен быть сохранен как переменная класса в другом классе.

Дескрипторы вызываются методом ```__getattribute__()```.
Переопределение ```__getattribute__()``` предотвращает автоматический вызов дескрипторов.

Посмотрите на следующий пример, определяющий дескриптор, который при обращении к нему записывает что-то в консоль:

In [2]:
# descriptors.py
class Verbose_attribute():
    """Класс дескриптора"""
    def __get__(self, obj, type=None) -> object:
        print("accessing the attribute to get the value")
        return 42
    def __set__(self, obj, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

class Foo():
    # экземпляр дескриптора
    attribute1 = Verbose_attribute()

my_foo_object = Foo()
x = my_foo_object.attribute1
print(x)

accessing the attribute to get the value
42


В приведенном выше примере Verbose_attribute() реализует протокол дескриптора. Как только она инстанцируется как атрибут Foo, ее можно считать дескриптором.

Когда к нему обращаются через ```.__get__()``` значения, он всегда возвращает значение 42.
Когда к нему обращаются, через ```.__set__()```, он вызывает исключение AttributeError, что является рекомендуемым способом реализации дескрипторов только для чтения.

Давайте, теперь посмотрим, как дескриптор может упростить наш программный код с обработкой координат точек.

Так как все координаты – целые числа, то интерфейс взаимодействия с ними мы определим через дескриптор с названием Integer (это имя мы, конечно же, придумываем сами):
```Py
class Integer:
    def __set_name__(self, owner, name):
        self.name = "_" + name
 
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
 
    def __set__(self, instance, value):
        print(f"__set__: {self.name} = {value}")
        instance.__dict__[self.name] = value
```
Пока не  обращаем внимания на его содержимое, сейчас я все подробно объясню. Затем, в классе Point3D мы создадим три атрибута как объекты класса Integer:
```Py
class Point3D:
    x = Integer()
    y = Integer()
    z = Integer()
 
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
```
Эти атрибуты и есть дескрипторы данных, через которые будет проходить взаимодействие. Итак, когда мы создавали экземпляры классов Integer, то автоматически вызывался магический метод ```__set_name__```, в котором параметр self являлся ссылкой на создаваемый экземпляр класса; owner – ссылка на класс Point3D; name – имя атрибута (для первого объекта x, затем, y и z). В этом методе мы формируем локальное свойство с именем атрибута, добавляя перед ним одно нижнее подчеркивание (так принято делать при определении дескрипторов). В итоге, в экземплярах классов будут храниться имена _x, _y, _z.


Зачем нам это нужно? Смотрите дальше. Предположим, мы создаем экземпляр класса Point3D:
```Py
pt = Point3D(1, 2, 3)
```
Сработает инициализатор, а в нем идет обращение к дескрипторам x, y, z. В частности, мы им присваиваем переданные значения. В этом случае, в классе Integer срабатывает сеттер (магический метод ```__set__```), параметр self – это ссылка на объект дескриптора; instance – ссылка на объект pt, из которого произошло обращение к дескриптору; value – присваиваемое значение. В этом сеттере мы выводим в консоль сообщение, что был вызван данный метод и отображаем сохраненное имя и присваиваемое значение. Следующей строчкой через ссылку instance, то есть, на экземпляр класса pt, формируем в нем локальное свойство с именем self.name и присваиваем значение value. В результате, в объекте pt появляются локальные свойства _x, _y, _z с соответствующими значениями.

Если затем, выполнить считывание данных через дескриптор, например, x, то автоматически сработает геттер (метод ```__get__```), в котором self – это ссылка на объект Integer; instance – ссылка на экземпляр класса pt; owner – ссылка на класс Point3D. Мы здесь через ссылку instance обращаемся к словарю ```__dict__``` и считываем значение нужного локального свойства, которое, затем, возвращается геттером. Это же значение автоматически возвращается и самим дескриптором.

Вот общая схема работы дескрипторов применительно к нашему классу Point3D. Теперь, сколько бы интерфейсов взаимодействия нам не понадобилось, мы легко их можем добавить в наш класс и все будет выглядеть понятно и компактно. На первый взгляд все это может показаться каким-то сложным и запутанным. Но, если внимательно во всем разобраться, то все предельно просто, только несколько громоздко

In [4]:
class Integer:
    def __set_name__(self, owner, name):
        self.name = "_" + name
 
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
 
    def __set__(self, instance, value):
        print(f"__set__: {self.name} = {value}")
        instance.__dict__[self.name] = value

class Point3D:
    x = Integer() # Наш интерфейс взаимодействия
    y = Integer()
    z = Integer()
 
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

pt = Point3D(1, 2, 3)

__set__: _x = 1
__set__: _y = 2
__set__: _z = 3


In [5]:
print(pt.__dict__)

{'_x': 1, '_y': 2, '_z': 3}


Последнее, что нужно прописать в дескрипторе – это проверку корректности данных. Для этого у нас уже есть метод verify_coord, перенесем его в класс Integer и вызовем в сеттере:

In [13]:
class Integer:
    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Координата должна быть целым числом")
 
    def __set_name__(self, owner, name):
        self.name = "_" + name
 
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
 
    def __set__(self, instance, value):
        self.verify_coord(value)
        instance.__dict__[self.name] = value

class Point3D:
    x = Integer()
    y = Integer()
    z = Integer()
 
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

pt = Point3D(1, 2, 3)
print(pt.__dict__)

{'_x': 1, '_y': 2, '_z': 3}


Теперь, если при формировании объекта указать неверный тип данных, то увидим сообщение об ошибке.
Еще в классе Integer я сделаю обращение к атрибутам экземпляра через стандартные функции getattr и setattr:
```Py
def __get__(self, instance, owner):
        return getattr(instance, self.name)

def __set__(self, instance, value):
    self.verify_coord(value)
    setattr(instance, self.name, value)
```
Так будет правильнее, с точки зрения Python, чем обращение напрямую к специальной коллекции ```__dict__```.


Резюмируем код:

In [51]:
class Integer:
    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Координата должна быть целым числом")
 
    def __set_name__(self, owner, name):
        self.name = "_" + name
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)

    def __set__(self, instance, value):
        self.verify_coord(value)
        setattr(instance, self.name, value)

class Point3D:
    x = Integer()
    y = Integer()
    z = Integer()
 
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

pt = Point3D(1, 2, 3)
print(pt.__dict__)

{'_x': 1, '_y': 2, '_z': 3}


В итоге, мы с вами определили дескриптор данных (data descriptor) и на его основе создали три объекта x, y, z для интерфейса взаимодействия с координатами точки объектов класса Point3D.

### Как осуществляется доступ к атрибутам с помощью цепочки поиска
Чтобы понять немного больше о дескрипторах Python и внутреннем устройстве Python, вам нужно понять, что происходит в Python при обращении к атрибуту. В Python каждый объект имеет встроенный атрибут ```__dict__```. Это словарь, который содержит все атрибуты, определенные в самом объекте. Чтобы увидеть это в действии, рассмотрим следующий пример:

In [24]:
class Vehicle():
    can_fly = False
    number_of_weels = 0

class Car(Vehicle):
    number_of_weels = 4

    def __init__(self, color):
        self.color = color

my_car = Car("red")
print(my_car.__dict__)
print(type(my_car).__dict__)

{'color': 'red'}
{'__module__': '__main__', 'number_of_weels': 4, '__init__': <function Car.__init__ at 0x107048ae0>, '__doc__': None}


Этот код создает новый объект и выводит содержимое атрибута ```__dict__``` как для объекта, так и для класса. Теперь запустите сценарий и проанализируйте вывод, чтобы увидеть установленные атрибуты ```__dict__```.

Атрибуты ```__dict__``` установлены, как и ожидалось. Обратите внимание, что в Python все является объектом. Класс - это тоже объект, поэтому у него также будет атрибут ```__dict__```, который содержит все атрибуты и методы класса.

Итак, что же происходит под капотом, когда вы обращаетесь к атрибуту в Python? Давайте проведем несколько тестов с помощью модифицированной версии предыдущего примера. Рассмотрим этот код:

In [25]:
# lookup.py
class Vehicle(object):
    can_fly = False
    number_of_weels = 0

class Car(Vehicle):
    number_of_weels = 4

    def __init__(self, color):
        self.color = color

my_car = Car("red")

print(my_car.color)
print(my_car.number_of_weels)
print(my_car.can_fly)

red
4
False


В этом примере вы создаете экземпляр класса Car, который наследуется от класса Vehicle. Затем вы получаете доступ к некоторым атрибутам. Если вы запустите этот пример, то увидите, что получите все ожидаемые значения.

Здесь, когда вы обращаетесь к атрибуту color экземпляра my_car, вы фактически обращаетесь к единственному значению атрибута ```__dict__``` объекта my_car. Когда вы обращаетесь к атрибуту number_of_wheels объекта my_car, вы на самом деле обращаетесь к единственному значению атрибута ```__dict__``` класса Car. Наконец, когда вы обращаетесь к атрибуту can_fly, вы на самом деле обращаетесь к нему с помощью атрибута ```__dict__``` класса Vehicle.

Это означает, что приведенный выше пример можно переписать следующим образом:

In [26]:
# lookup2.py
class Vehicle():
    can_fly = False
    number_of_weels = 0

class Car(Vehicle):
    number_of_weels = 4

    def __init__(self, color):
        self.color = color

my_car = Car("red")

print(my_car.__dict__['color'])
print(type(my_car).__dict__['number_of_weels'])
print(type(my_car).__base__.__dict__['can_fly']) # TYPE.__BASE__ Указывает на ближайший родительский тип, определяющий для текущего его основную линию наследственности.

red
4
False


Что же происходит, когда вы обращаетесь к атрибуту объекта с помощью точечной нотации? Как интерпретатор узнает, что вам действительно нужно? Здесь вступает в силу концепция, называемая цепочкой поиска (lookup chain): **ЦЕПОЧКА ПОИСКА**. **ПРИОРИТЕТ ПОИСКА** **PY**

- Сначала вы получите результат, возвращенный методом ```__get__``` дескриптора данных, названного по имени искомого атрибута.

- Если это не удается, то вы получите значение ```__dict__``` вашего объекта для ключа, названного по имени искомого атрибута.

- Если это не удается, то вы получите результат, возвращенный методом ```__get__``` дескриптора не-данных, названного по имени искомого атрибута.

- Если это не удается, то вы получите значение ```__dict__``` вашего типа объекта для ключа, названного по имени искомого атрибута.

- Если это не удастся, то вы получите значение ```__dict__``` родительского типа вашего объекта для ключа, названного после искомого атрибута.

- Если это не удалось, то предыдущий шаг повторяется для всех родительских типов в порядке разрешения методов вашего объекта. (mro)

- Если все остальное не удалось, то вы получите исключение AttributeError.

Теперь вы понимаете, почему важно знать, является ли дескриптор дескриптором данных или дескриптором не данных? Они находятся на разных уровнях цепочки поиска, и позже вы увидите, что эта разница в поведении может быть очень удобной.

In [27]:
class Ten:
    def __get__(self, obj, objtype=None):
        return 10

class A:
    x = 5                       # Regular class attribute
    y = Ten()                   # Descriptor instance

a = A()                     # Make an instance of class A
a.x                         # Normal attribute lookup

a.y                         # Descriptor lookup

10

При поиске атрибута a.x оператор точки находит 'x': 5 в словаре класса. В поиске атрибута a.y оператор точки находит экземпляр дескриптора, распознанный по его методу ```__get__```. Вызов этого метода возвращает 10.

Теперь вернемся к тому, что вначале мы говорили, что важно различать дескрипторы данных и не данных. Напоминание: (Если ваш дескриптор реализует только ```.__get__()```, то говорят, что это **non-data descriptor** (дескриптор не данных). Если он реализует ```.__set__()``` или ```.__delete__()```, то говорят, что это **data descriptor** (дескриптор данных). Обратите внимание, что разница не только в названии, но и в поведении. Это потому, что дескрипторы данных имеют приоритет в процессе поиска) В чем разница? Ну, во-первых, очевидно, дескрипторы не данных не могут менять значения какого-либо свойства, так как не имеют сеттера и делитера. **Они служат только для считывания информации**. И есть второе важное отличие. **приоритет доступа.** О чем здесь речь? О lookup chain Давайте я покажу это различие на примере. Добавим в программу еще один дескриптор, только не данных (**non-data descriptor**):
```Py
class ReadIntX:
    def __set_name__(self, owner, name):
        self.name = "_x"
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)
```
Он у нас будет считывать локальное свойство _x. Определим его в классе Point3D:
```Py
xr = ReadIntX()
```
И теперь можем использовать для считывания локального атрибута _x:
```Py
print(pt.xr)
```

Т.e: Если в ```__dict__``` владельца имеется запись с ключём, совпадающим с именем атрибута, указывающего на дескриптор, то в случае дескрипторов не-данных при обращении к атрибуту вернётся значение из ```__dict__```, а не из дескриптора. В случае дескрипторов данных наоборот — значение из дескриптора.

In [19]:
class Integer:
    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Координата должна быть целым числом")
 
    def __set_name__(self, owner, name):
        self.name = "_" + name
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)

    def __set__(self, instance, value):
        self.verify_coord(value)
        setattr(instance, self.name, value)

class ReadIntX: # <-- non-data descriptor
    def __set_name__(self, owner, name):
        self.name = "_x"
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)
    
class Point3D:
    x = Integer()
    y = Integer()
    z = Integer()
    xr = ReadIntX() # <-- non-data descriptor instance
 
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

pt = Point3D(1, 2, 3)
print(pt.__dict__)
##
pt.xr = 5
print(pt.xr, pt.__dict__) # pt.xr возв

{'_x': 1, '_y': 2, '_z': 3}
5 {'_x': 1, '_y': 2, '_z': 3, 'xr': 5}


Как видите, все работает. Но, что будет, если мы запишем конструкцию:
```Py
pt.xr = 5
```
Произойдет ошибка? Нет! В экземпляре pt будет создано новое локальное свойство с именем xr и мы в этом можем убедиться:
```Py
print(pt.xr, pt.__dict__)
```
Кроме того, при обращении к pt.xr мы получаем значение 5, а не 1. Это, как раз и говорит о том, что приоритет доступа к локальным свойствам объекта и к дескриптору не данных одинаков. (Код выше)

Однако, если в дескриптор добавить сеттер и превратить его в дескриптор данных:
```Py
class ReadIntX:
    def __set_name__(self, owner, name):
        self.name = "_x"
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)
 
    def __set__(self, instance, value):
        setattr(instance, self.name, value)
```
А создание локального атрибута xr в объекте pt мы сделаем через коллекцию ```__dict__```:
```Py
pt.__dict__['xr'] = 5
```
то при выполнении:
```Py
print(pt.xr, pt.__dict__)
```
увидим значение 1, хотя в объекте существует свойство xr, которое установили в значение 5. Это произошло потому, что **приоритет обращению к дескриптору данных выше, чем к локальным атрибутам экземпляра класса**. То есть, здесь все работает ровно так, как и с доступом к объектам-свойствам, о которых мы говорили на прошлых занятиях:

In [20]:
class Integer:
    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Координата должна быть целым числом")
 
    def __set_name__(self, owner, name):
        self.name = "_" + name
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)

    def __set__(self, instance, value):
        self.verify_coord(value)
        setattr(instance, self.name, value)

class ReadIntX: # <-- data descriptor
    def __set_name__(self, owner, name):
        self.name = "_x"
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)
 
    def __set__(self, instance, value):
        setattr(instance, self.name, value)
    
class Point3D:
    x = Integer()
    y = Integer()
    z = Integer()
    xr = ReadIntX() # <-- data descriptor instance
 
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

pt = Point3D(1, 2, 3)
print(pt.__dict__)
##
#pt.xr = 5
#print(pt.xr, pt.__dict__)
pt.__dict__['xr'] = 5
print(pt.xr, pt.__dict__)

{'_x': 1, '_y': 2, '_z': 3}
1 {'_x': 1, '_y': 2, '_z': 3, 'xr': 5}


Еще один важный момент: дескрипторы Python инстанцируются только один раз для каждого класса. Это означает, что каждый экземпляр класса, содержащего дескриптор, разделяет этот экземпляр дескриптора. Это то, чего вы можете не ожидать, и может привести к классической ошибке, например, такой:

In [3]:
# descriptors2.py
class OneDigitNumericValue():
    # set name поможет исправить ситуацию, как в примерах выше
    def __init__(self):
        self.value = 0
    def __get__(self, obj, type=None) -> object:
        return self.value
    def __set__(self, obj, value) -> None:
        if value > 9 or value < 0 or int(value) != value:
            raise AttributeError("The value is invalid")
        self.value = value

class Foo():
    number = OneDigitNumericValue()

my_foo_object = Foo()
my_second_foo_object = Foo()

my_foo_object.number = 3
print(my_foo_object.number) # ожидали 0
print(my_second_foo_object.number)

my_third_foo_object = Foo()
print(my_third_foo_object.number)

3
3
3


With set name:

С помощью этого нового метода каждый раз, когда вы инстанцируете дескриптор, вызывается этот метод и автоматически устанавливается параметр name.

In [52]:
# descriptors5.py
class OneDigitNumericValue():
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, type=None) -> object:
        return obj.__dict__.get(self.name) or 0

    def __set__(self, obj, value) -> None:
        obj.__dict__[self.name] = value

class Foo():
    number = OneDigitNumericValue()

my_foo_object = Foo()
my_second_foo_object = Foo()

my_foo_object.number = 3
print(my_foo_object.number)
print(my_second_foo_object.number)

my_third_foo_object = Foo()
print(my_third_foo_object.number)

3
0
0


Вернемся к примеру:

In [22]:
# descriptors.py
class Verbose_attribute():
    def __get__(self, obj, type=None) -> object:
        print("accessing the attribute to get the value")
        return 42
    def __set__(self, obj, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

class Foo():
    attribute1 = Verbose_attribute()

my_foo_object = Foo()
x = my_foo_object.attribute1
print(x)

accessing the attribute to get the value
42


В приведенном выше примере Verbose_attribute() реализует протокол дескриптора. Как только она инстанцируется как атрибут Foo, ее можно считать дескриптором.

Когда к нему обращаются через ```.__get__()``` значения, он всегда возвращает значение 42.
Когда к нему обращаются, через ```.__set__()```, он вызывает исключение AttributeError, что является рекомендуемым способом реализации дескрипторов только для чтения.

Интересно, но свойства (@property) в Python - это просто.. дескрипторы!

Если вы хотите получить тот же результат, что и в предыдущем примере, без явного использования дескриптора Python, то наиболее простым подходом будет использование свойства (property). В следующем примере используется свойство, которое при обращении к нему записывает сообщение в консоль:

In [21]:
# property_decorator.py
class Foo():
    @property
    def attribute1(self) -> object:
        print("accessing the attribute to get the value")
        return 42

    @attribute1.setter
    def attribute1(self, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

my_foo_object = Foo()
x = my_foo_object.attribute1
print(x)

accessing the attribute to get the value
42


В приведенном выше примере декораторы используются для определения свойства с методами getter и setter. Но, как мы знаем, декораторы - это просто синтаксический сахар. На самом деле, приведенный пример можно записать следующим образом:

In [23]:
# property_function.py
class Foo():
    def getter(self) -> object:
        print("accessing the attribute to get the value")
        return 42

    def setter(self, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

    attribute1 = property(getter, setter)

my_foo_object = Foo()
x = my_foo_object.attribute1
print(x)

accessing the attribute to get the value
42


Теперь вы видите, что свойство было создано с помощью функции property(). Напомним шаблон этой функции:
```
property(fget=None, fset=None, fdel=None, doc=None) -> object
```

property() возвращает объект-свойство, реализующий протокол дескриптора. Он использует параметры fget, fset и fdel для фактической реализации трех методов протокола.

В заключение.
Дескриптор - это то, как мы называем любой объект, который определяет ```__get__()```, ```__set__()``` или ```__delete__()```.

Как вариант, дескрипторы могут иметь метод ```__set_name__()```. Он используется только в тех случаях, когда дескриптору необходимо знать либо класс, в котором он был создан, либо имя переменной класса, которой он был присвоен. (Этот метод, если он присутствует, вызывается, даже если класс не является дескриптором).

Дескрипторы вызываются оператором dot (точка) во время поиска атрибутов. Если к дескриптору обращаются косвенно с помощью vars(some_class)[имя_дескриптора], то экземпляр дескриптора возвращается без его вызова.

Дескрипторы работают только при использовании в качестве переменных класса. Когда они помещены в экземпляры, они не имеют никакого эффекта.

Основная мотивация дескрипторов - предоставить "крючок", позволяющий объектам, хранящимся в переменных класса, управлять тем, что происходит во время поиска атрибутов.

Традиционно вызывающий класс управляет тем, что происходит во время поиска. Дескрипторы меняют эти отношения и позволяют данным, которые ищутся, иметь право голоса.

Дескрипторы используются во всем языке. С их помощью функции превращаются в связанные методы. Такие распространенные инструменты, как classmethod(), staticmethod(), property() и functools.cached_property(), реализованы в виде дескрипторов.

Хоть и этот функционал относительно редко используется в практике. Но знать это необходимо, чтобы при необходимости не изобретать велосипед, а использовать уже встроенные возможности языка Python.